In [1]:
#import libraries
import numpy as np
import pandas as pd

In [2]:
# Load the data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
train_data.head(2)

,UID,col_0,col_1,col_2,col_3,col_4,col_5,col_6,y
0,0,A 0,B0,C2,D1,100,E1,F2,237000
1,1,A1,B0,C11,D4,100,E4,F2,86193


In [4]:
train_data.describe

<bound method NDFrame.describe of        UID col_0   col_1 col_2 col_3  col_4 col_5 col_6       y
0        0   A 0      B0    C2    D1    100    E1    F2  237000
1        1    A1      B0   C11    D4    100    E4    F2   86193
2        2    A0      B0   C18    D0      0    E0    F2  169200
3        3    A2      B0   C11    D1    100    E1    F2   58000
4        4    A0      B0   C67    D1      0    E1    F2  235000
...    ...   ...     ...   ...   ...    ...   ...   ...     ...
2623  2623    A1      B0    C2    D1      0    E1    F2  102100
2624  2624    A0   B0       C8    D1      0    E1    F2  129300
2625  2625    A0      BO    C7    D1    100    E1    F2  275300
2626  2626    A0      B0   C11    D1    100    E1    F2  150000
2627  2627    A0      B0    C2    D1    100    E1    F2  191475

[2628 rows x 9 columns]>

In [5]:
# Extract features and target
X_train = train_data.iloc[600:, 1:-1]
y_train = train_data.iloc[600:, -1]
X_val = train_data.iloc[:600, 1:-1]
y_val = train_data.iloc[:600, -1]
X_test = test_data.iloc[:, 1:]

In [6]:
X_train.isnull().sum()

col_0    109
col_1     65
col_2      0
col_3      0
col_4      0
col_5      0
col_6     88
dtype: int64

In [7]:
X_train.head(2)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6
600,A0,B0,C50,D1,0,E1,F2
601,A0,B0,C2,D1,100,E1,F2


In [8]:
unique_count=X_train["col_4"].unique()
print(unique_count)

[  0 100  50]


In [9]:
# Categorical columns
categorical_cols = ['col_0', 'col_1', 'col_2', 'col_3', 'col_5', 'col_6']

In [10]:
# Fill NaN values in categorical columns with the most frequent value in the respective columns
def fill_na(data, categorical_cols):
    for col in categorical_cols:
        most_frequent_value = data[col].mode().iloc[0]  # Get the mode (most frequent value)
        data[col].fillna(most_frequent_value, inplace=True)
    return data

X_train = fill_na(X_train, categorical_cols)
X_val = fill_na(X_val, categorical_cols)
X_test = fill_na(X_test, categorical_cols)

In [11]:
#one hot encoding
def one_hot_encoding(data,categorical_cols):
    for col in categorical_cols:
        unique_categories = data[col].unique()
        for category in unique_categories:
            new_col_name = f"{col}_{category}"
            data[new_col_name] = (data[col] == category).astype(int)

    # Drop the original categorical columns from data
    data.drop(columns=categorical_cols, inplace=True)
    return data

X_train = one_hot_encoding(X_train, categorical_cols)
X_val = one_hot_encoding(X_val, categorical_cols)
X_test = one_hot_encoding(X_test, categorical_cols)

C:\Users\klikh\AppData\Local\Temp\ipykernel_8604\2338484223.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = (data[col] == category).astype(int)
C:\Users\klikh\AppData\Local\Temp\ipykernel_8604\2338484223.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = (data[col] == category).astype(int)
C:\Users\klikh\AppData\Local\Temp\ipykernel_8604\2338484223.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

In [12]:
# Align columns to ensure they have the same features
common_columns = list(set(X_train.columns) & set(X_test.columns)& set(X_val.columns))
X_train = X_train[common_columns]
X_test = X_test[common_columns]
X_val = X_val[common_columns]

In [13]:
# normalize
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

X_train['col_4'] = normalize(X_train['col_4'] )
X_val['col_4']  = normalize(X_val['col_4'] )
X_test['col_4']  = normalize(X_test['col_4'] )


In [14]:
# convert df into numpy
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()
X_test = X_test.to_numpy()

In [15]:
# hyperparameters
learning_rate = 0.01
epochs = 15000
lambda_reg = 0.1


In [16]:
# weights and biases
w = np.zeros(X_train.shape[1])
b = 0

# Early stopping parameters
patience = 50 
best_val_rmse = float("inf")
best_weights = None
best_epoch = 0

In [17]:
# Prediction 
def predict(X_test, weights, bias):
    return np.dot(X_test, weights) + bias


In [18]:
# rmse function
def calculate_rmse(y_true, y_pred):
    squared_errors = (y_true - y_pred) ** 2
    mean_squared_error = np.mean(squared_errors)
    rmse = np.sqrt(mean_squared_error)
    return rmse   

In [19]:
# Training loop
for epoch in range(epochs):
    # Forward pass
    predictions = np.dot(X_train, w) + b

    # loss (MSE)
    loss = np.mean(0.5 * (predictions - y_train) ** 2) + (lambda_reg / (2 * X_train.shape[0])) * np.sum(w ** 2)

    # gradients
    dw = (1 / X_train.shape[0]) * (np.dot(X_train.T, (predictions - y_train)) + lambda_reg * w)
    db = (1 / X_train.shape[0]) * np.sum(predictions - y_train)

    # Update weights and bias
    w -= learning_rate * dw
    b -= learning_rate * db

    # Monitoring loss
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

    # Check validation RMSE for early stopping
    if (epoch + 1) % 10 == 0:
        val_predictions = predict(X_val, w, b)
        val_rmse = calculate_rmse(y_val, val_predictions)

        # If validation RMSE is the best so far, update the best RMSE and weights
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            best_weights = w.copy()
            best_epoch = epoch + 1

        # If there's no improvement for 'patience' epochs, stop training
        if epoch + 1 - best_epoch >= patience:
            print(f"Early stopping at epoch {epoch + 1} with best validation RMSE: {best_val_rmse:.4f}")
            break


Epoch 99, Loss: 1509656770.5362
Epoch 199, Loss: 1460588464.1236
Epoch 299, Loss: 1424189990.9762
Epoch 399, Loss: 1395722859.8174
Epoch 499, Loss: 1372686977.5266
Epoch 599, Loss: 1353534934.1886
Epoch 699, Loss: 1337267444.7300
Epoch 799, Loss: 1323212489.7183
Epoch 899, Loss: 1310901124.9864
Epoch 999, Loss: 1299995287.8829
Epoch 1099, Loss: 1290244313.8241
Epoch 1199, Loss: 1281457790.2325
Epoch 1299, Loss: 1273488018.1895
Epoch 1399, Loss: 1266218319.8667
Epoch 1499, Loss: 1259555023.4708
Epoch 1599, Loss: 1253421834.8377
Epoch 1699, Loss: 1247755801.7882
Epoch 1799, Loss: 1242504367.6510
Epoch 1899, Loss: 1237623185.3732
Epoch 1999, Loss: 1233074472.4433
Epoch 2099, Loss: 1228825756.4567
Epoch 2199, Loss: 1224848906.8209
Epoch 2299, Loss: 1221119378.7314
Epoch 2399, Loss: 1217615616.4759
Epoch 2499, Loss: 1214318577.6604
Epoch 2599, Loss: 1211211350.1748
Epoch 2699, Loss: 1208278841.0025
Epoch 2799, Loss: 1205507521.2201
Epoch 2899, Loss: 1202885215.3521
Epoch 2999, Loss: 1200400

In [20]:

# Use the best weights for prediction
best_predictions = predict(X_test, best_weights, b)
result_df = pd.DataFrame({'UID': test_data['UID'], 'y': best_predictions})
# Save predictions 
result_df.to_csv('Predictions.csv', index=False)